<a href="https://colab.research.google.com/github/bohjort/AI-ML-adventure/blob/main/OMX30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [3]:
CSV_COLUMN_NAMES = ['Year', 'Month', 'Day','Weekday','High', 'Low', 'Close','High_ch_pc_sdbf', 'Low_ch_pc_sdbf', 'Close_ch_pc_sdbf', 'High_ch_tot_sdbf', 'Low_ch_tot_sdbf', 'Close_ch_tot_sdbf','Streak','Bestmove']
Bestmove = ['Salj', 'Kop']
# Lets define some constants to help us later on

In [4]:
url1 = 'https://raw.githubusercontent.com/bohjort/AI-ML-adventure/main/eval-20200901.csv'
url2 = 'https://raw.githubusercontent.com/bohjort/AI-ML-adventure/main/omx30-20200901-1y.csv'
url3 = 'https://raw.githubusercontent.com/bohjort/AI-ML-adventure/main/omx30training.csv'
url4 = 'https://raw.githubusercontent.com/bohjort/AI-ML-adventure/main/OMX30trainingdata.csv'
url5 = 'https://raw.githubusercontent.com/bohjort/AI-ML-adventure/main/OMX30testingdata.csv'
url6 = 'https://raw.githubusercontent.com/bohjort/AI-ML-adventure/main/OMX30trainingdata2.csv'



In [5]:
df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df3 = pd.read_csv(url6)
df4 = pd.read_csv(url5)

In [6]:
print(df3)
print(df4)

     Year  Month  Day  ...  Close_ch_tot_sdbf  Streak  Bestmove
0    2021      9   30  ...           3.630000       2         0
1    2021      9   29  ...           1.530000       1         1
2    2021      9   28  ...         -49.990000      -3         1
3    2021      9   27  ...          -9.260000      -2         0
4    2021      9   24  ...         -24.730000      -1         0
..    ...    ...  ...  ...                ...     ...       ...
353  2020      5    8  ...          25.450000       2         0
354  2020      5    7  ...          16.340000       1         1
355  2020      5    6  ...         -16.780000      -1         1
356  2020      5    5  ...          32.970000       1         0
357  2020      5    4  ...           2.200462      -1         1

[358 rows x 15 columns]
    Year  Month  Day  ...  Close_ch_tot_sdbf  Streak  Bestmove
0   2021     10   20  ...              -0.41      -1         0
1   2021     10   19  ...              14.96       1         0
2   2021     10   

In [7]:
train = df3
test = df4
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [8]:
train_y = train.pop('Bestmove')
test_y = test.pop('Bestmove')
train.head() # the Bestmove column is now gone

,Year,Month,Day,Weekday,High,Low,Close,High_ch_pc_sdbf,Low_ch_pc_sdbf,Close_ch_pc_sdbf,High_ch_tot_sdbf,Low_ch_tot_sdbf,Close_ch_tot_sdbf,Streak
0,2021,9,30,3,2278.19,2249.46,2259.18,0.001264,-0.000325,0.001607,2.88,-0.73,3.63,2
1,2021,9,29,2,2275.31,2250.19,2255.55,-0.010781,-0.001542,0.000678,-24.53,-3.47,1.53,1
2,2021,9,28,1,2299.84,2253.66,2254.02,-0.014701,-0.020819,-0.022178,-33.81,-46.92,-49.99,-3
3,2021,9,27,0,2333.65,2300.58,2304.01,0.000544,-0.003708,-0.004019,1.27,-8.53,-9.26,-2
4,2021,9,24,4,2332.38,2309.11,2313.27,-0.003310,-0.003413,-0.010690,-7.72,-7.88,-24.73,-1


In [9]:
def input_fn(features, labels, training=True, batch_size=32):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [10]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='Year', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Month', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Day', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Weekday', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='High', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Low', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Close', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='High_ch_pc_sdbf', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Low_ch_pc_sdbf', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Close_ch_pc_sdbf', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=

In [11]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[1024,512,256],
    # The model must choose between 3 classes.
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpt9r3n9x9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=20000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpt9r3n9x9/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 9.941675, step = 0
INFO:tensorflow:global_step/sec: 78.8841
INFO:tensorflow:loss = 5.9343443, step = 100 (1.270 sec)
INFO:tensorflow:global_step/sec: 86.2877
INFO:tensorflow:loss = 1.1625221, step = 200 (1.159 sec)
INFO:tensorflow:global_step/sec:

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
#@title
def input_fn(features, batch_size=100):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['Year', 'Month', 'Day','Weekday','High', 'Low', 'Close','High_ch_pc_sdbf', 'Low_ch_pc_sdbf', 'Close_ch_pc_sdbf', 'High_ch_tot_sdbf', 'Low_ch_tot_sdbf', 'Close_ch_tot_sdbf','Streak']
predict = {'Year': [2021],'Month': [10],'Day': [25],'Weekday': [0],'High': [2353.63],'Low': [2326.72],'Close': [2330.13],'High_ch_pc_sdbf': [0.000675552232084119],'Low_ch_pc_sdbf': [0.00453857791225414],'Close_ch_pc_sdbf': [-0.00802959491530517],'High_ch_tot_sdbf': [1.59000000000015],'Low_ch_tot_sdbf': [10.5599999999999],'Close_ch_tot_sdbf': [-18.71],'Streak': [-1]}

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        Bestmove[class_id], 100 * probability))